In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
# 예찬 _ nltk에 저장되어 있는 stop word를 받기 위한 것들
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\park\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\park\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# 텍스트 정규화
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\park\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [4]:
data = pd.read_csv('test.csv', engine ='python') # csv파일 올려서 사용
data.head(1)

,학수번호,과목명,수업년도,수업학기,인문사회,자연과학,공학,예체능,국문개요,영문개요,키워드,추천강의,Unnamed: 12,Unnamed: 13
0,ACC2051,중급회계1,2020.0,1.0,267.0,0.0,3.0,5.0,ACC 251 재무회계 기업 재무회계의 이론적 구조를 이해하고 일반적으로 인정된 회...,Acc 251 Financial Accounting This course is in...,"회계, 재무회계, 회계정보, 의사, 신용의사","회계정보의활용, 기업윤리와사회적책임, 회계정보의이해, 스크린영어, 보이스앙상블",NaN,NaN


In [5]:
# overview에서 Null 값을 가진 경우에는 Null 값을 제거
data['영문개요'] = data['영문개요'].fillna('')

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
# 국문개요에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['영문개요'])
print(tfidf_matrix.shape)

(309, 2578)


In [7]:
# 예찬 _ 영문 개요를 word_tokenize를 통해서 문장을 각 단어들로 쪼개서 저장함
words = word_tokenize(data['영문개요'][0])
print(words)

['Acc', '251', 'Financial', 'Accounting', 'This', 'course', 'is', 'intermediate', 'accounting', '.', 'It', 'covers', 'basic', 'theories', ',', 'conceptual', 'frameworks', 'of', 'financial', 'statements', 'involving', 'assets', ',', 'liabilities', ',', 'and', 'owner', "'s", 'equity', '.', 'This', 'course', 'improves', 'ability', 'for', 'students', 'to', 'make', 'investment', 'and', 'credit', 'decision', ',', 'to', 'analyze', 'future', 'cash-flow', 'in', 'business', '.']


In [8]:
filtered_words = []
# 예찬 _ nltk에 있는 stopword를 필터해서 새로 저장함
stopwords = nltk.corpus.stopwords.words('english')

for word in words:
    if word not in stopwords:
        # 예찬 _ 새로운 filtered_words에 filter를 마친 단어들을 저장함
        filtered_words.append(word)
        
print(filtered_words)

['Acc', '251', 'Financial', 'Accounting', 'This', 'course', 'intermediate', 'accounting', '.', 'It', 'covers', 'basic', 'theories', ',', 'conceptual', 'frameworks', 'financial', 'statements', 'involving', 'assets', ',', 'liabilities', ',', 'owner', "'s", 'equity', '.', 'This', 'course', 'improves', 'ability', 'students', 'make', 'investment', 'credit', 'decision', ',', 'analyze', 'future', 'cash-flow', 'business', '.']


In [19]:
# 예찬 _ 의도하고 싶은 것 : 전체 영문개요의 내용을 일괄적으로 stop word를 이용해서 필터하고 그 내용을 가지고 코사인 유사도를 구하고 싶다 !
filtered_words = [' ' for _ in range(len(data['영문개요']))]
stopwords = nltk.corpus.stopwords.words('english')
# 예찬 _ 정규화 작업을 위해서 WordNetLemmatizer()호ㅓ출
lemma = WordNetLemmatizer()
for i in range(len(data['영문개요'])):
  words = word_tokenize(data['영문개요'][i])
  # print(words)
  for word in words:
    if word not in stopwords:
        # 예찬 _ word에 속한 단어를 명사로 변환
        word = lemma.lemmatize(word, 'n')
        # 예찬 _ 새로운 filtered_words에 filter를 마친 단어들을 저장함
        filtered_words[i] += (word+' ')
print(filtered_words)
# print(len(filtered_words))

[" Acc 251 Financial Accounting This course intermediate accounting . It cover basic theory , conceptual framework financial statement involving asset , liability , owner 's equity . This course improves ability student make investment credit decision , analyze future cash-flow business . ", ' artificial intelligence ', ' Introduction Deep Learning ', ' R-Py computing ', ' Machine Learning ', ' BUS 105 Introductory Statistics Business The course discus role statistic play decision-making process . It also cover basic principle step statistical analysis probability , probability distribution , sampling distribution , hypothesis testing ', ' Bus 1010 Understanding Corporate Accounting Information The objective course teach basic concept principle accounting beginning student accounting . The main content course consist role accounting information modern business , accounting standard financial statement . ', " The purpose course develop student ' understanding mathematics widely used mak

In [20]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:
indices = pd.Series(data.index, index=data['과목명']).drop_duplicates()
print(indices.head())

과목명
중급회계1           0
AI+X:인공지능       1
AI+X:딥러닝        2
AI+X:R-Py컴퓨팅    3
AI+X:머신러닝       4
dtype: int64


In [22]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [23]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 국문개요에 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 과목에 대해서 해당 과목와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 과목를 받아옵니다.
    sim_scores = sim_scores[1:6]

    # 가장 유사한 10개의 과목의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['과목명'].iloc[movie_indices]

In [28]:
get_recommendations('실용중국어')

128       중급중국어
149     한류와한국문화
254    한국의문화와역사
281      C프로그래밍
201       교양태권도
Name: 과목명, dtype: object

In [ ]:
print(indices['현대사회와페미니즘'])
print(indices['선택형Co-op'])
print(indices.index[255])

255
307
현대사회와페미니즘


In [ ]:
i=255
while i<308:
  print('과목명: ', indices.index[i])
  print(get_recommendations(indices.index[i]))
  print('============================')
  i=i+1

  